In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.values, wine.target, stratify=wine.target, test_size=0.2, random_state=0)

In [4]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

print(X_train.shape)
print(y_train.shape)

torch.Size([142, 13])
torch.Size([142])


In [5]:
train = TensorDataset(X_train, y_train)

print(train[0])

# minibiatch
train_loader = DataLoader(train, batch_size=16, shuffle=True)

(tensor([1.1840e+01, 8.9000e-01, 2.5800e+00, 1.8000e+01, 9.4000e+01, 2.2000e+00,
        2.2100e+00, 2.2000e-01, 2.3500e+00, 3.0500e+00, 7.9000e-01, 3.0800e+00,
        5.2000e+02]), tensor(1))


In [6]:
for xb, yb in train_loader:
    print(xb.shape)
    print(yb.shape)
    print(xb)
    print("-"*8)
    print(yb)
    break

torch.Size([16, 13])
torch.Size([16])
tensor([[1.2160e+01, 1.6100e+00, 2.3100e+00, 2.2800e+01, 9.0000e+01, 1.7800e+00,
         1.6900e+00, 4.3000e-01, 1.5600e+00, 2.4500e+00, 1.3300e+00, 2.2600e+00,
         4.9500e+02],
        [1.3290e+01, 1.9700e+00, 2.6800e+00, 1.6800e+01, 1.0200e+02, 3.0000e+00,
         3.2300e+00, 3.1000e-01, 1.6600e+00, 6.0000e+00, 1.0700e+00, 2.8400e+00,
         1.2700e+03],
        [1.2290e+01, 2.8300e+00, 2.2200e+00, 1.8000e+01, 8.8000e+01, 2.4500e+00,
         2.2500e+00, 2.5000e-01, 1.9900e+00, 2.1500e+00, 1.1500e+00, 3.3000e+00,
         2.9000e+02],
        [1.3880e+01, 1.8900e+00, 2.5900e+00, 1.5000e+01, 1.0100e+02, 3.2500e+00,
         3.5600e+00, 1.7000e-01, 1.7000e+00, 5.4300e+00, 8.8000e-01, 3.5600e+00,
         1.0950e+03],
        [1.2220e+01, 1.2900e+00, 1.9400e+00, 1.9000e+01, 9.2000e+01, 2.3600e+00,
         2.0400e+00, 3.9000e-01, 2.0800e+00, 2.7000e+00, 8.6000e-01, 3.0200e+00,
         3.1200e+02],
        [1.2670e+01, 9.8000e-01, 2.2400e+0

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net()

In [14]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10000):
    total_loss = 0
    for batch_x, batch_y in train_loader:
#         batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

50 4.822485536336899
100 4.855052590370178
150 4.797134816646576
200 4.846905499696732
250 4.629577130079269
300 5.394599825143814
350 5.591883987188339
400 4.243719905614853
450 4.976611107587814
500 4.305519491434097
550 5.489367514848709
600 4.5565899312496185
650 4.427371293306351
700 5.065387651324272
750 3.8414291441440582
800 5.139560103416443
850 4.542719095945358
900 4.043749451637268
950 3.752570331096649
1000 4.432940661907196
1050 3.905428186058998
1100 5.21517539024353
1150 4.360669031739235
1200 4.086588010191917
1250 3.609330877661705
1300 3.3422208428382874
1350 3.51890105009079
1400 3.2217068821191788
1450 4.798635944724083
1500 3.5261503756046295
1550 3.316948801279068
1600 4.4452371299266815
1650 3.855015143752098
1700 5.137949675321579
1750 3.778383180499077
1800 4.3511693328619
1850 2.962446630001068
1900 3.1771743297576904
1950 3.15942145884037
2000 4.0534925907850266
2050 3.218295633792877
2100 2.7191073298454285
2150 3.093681752681732
2200 3.62271910905838
2250 

In [15]:
X_test, y_test = Variable(X_test), Variable(y_test)
result = torch.max(model(X_test).data, 1)[1]

In [16]:
y_test.data.numpy() == result.numpy()

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True])

In [17]:
accuracy = sum(y_test.data.numpy() == result.numpy()) / len(y_test.data.numpy())
accuracy

0.8611111111111112